## Zadania: LangGraph - tool node, conditional edge, loop


Zadanie 1 \
Tool node: word_count \
Zmodyfikuj poniższą funkcje narzędzia tak aby zliczała litery zamiast wyrazów.

In [1]:
def word_count_tool(state: dict) -> dict:
    q = state.get("question","")
    state["wc"] = len(str(q).split())
    return state
assert word_count_tool({"question":"Ala ma kota"})["wc"] == 3


Zadanie 2 \
Router: warunek dalszej pracy \
Przeanalizuj poniższy kod. \
Narysuj graf budowany za pomocą poniższego kodu. Jaki jest warunek przejścia do węzła końcowego?

In [3]:
from typing import TypedDict, Optional
try:
    from langgraph.graph import StateGraph, START, END
except Exception as e:
    print("LangGraph nie jest zainstalowany (demo).", e)
    StateGraph = object
    START="__START__"
    END="__END__"

class State(TypedDict, total=False):
    question: str
    wc: int
    answer: Optional[str]
    loops: int

def generator_node(state: State) -> State:
    if state.get("wc",0) > 4:
        state["answer"] = "Dłuższa odpowiedź..."
    else:
        state["answer"] = "Krótka odpowiedź."
    return state

def need_more_detail(state: State) -> bool:
    return state.get("answer") == "Krótka odpowiedź."

builder = StateGraph(State)
try:
    builder.add_node("tool_wc", word_count_tool)
    builder.add_node("gen", generator_node)
    builder.add_edge(START, "tool_wc")
    builder.add_edge("tool_wc", "gen")
except Exception as e:
    print("Szkic dodawania węzłów/krawędzi:", e)

def router(state: State) -> str:
    return "loopback" if need_more_detail(state) else END

try:
    builder.add_conditional_edges("gen", router, {"loopback": "tool_wc", END: END})
except Exception as e:
    print("Szkic add_conditional_edges:", e)
